In [2]:
# Comment Line 

import math
import numpy as np
import matplotlib.pyplot as plt
import scipy

# First we need to define the lattice length
# next multiply by the a constant to convert to angstrom
# Nums is the number of atoms
alat = 29.00
alat_ang = alat*0.529
nums = list(range(257))

atom_num = 256
dist = []
r = 0
cluster = []
averages = []
columb_force = []

# Coulumb Constant : Units : N m^2 C^-2
colum = 9.0*10**29
e2 = (1.602*10**-19)**2


n = 0

# lattice lengths a for x, b for y, c for z
a = 15.341
b = 15.341
c = 15.341

cutoff = 3.5

newfilename = 'sep23_1_atom_displacement'

######
# READ IN FILE , USE SPLIT TO SEPERATE OUT ALL VALUES
######

# Open file, proceed to write out a new file where atom names are replaced so that Lammps can read it

with open("sep23.1_final_structure", "rt") as fin:
    with open("atom_pos_temp", "wt") as fout: 
        for line in fin: 
            fout.write((line.replace('Pt', '1')).replace('Ni','2').replace('P','3'))
    
with open('atom_pos_temp') as f:
    alines = f.read().split()

######
# Use float to convert to float numbers
######
xpos = []
ypos = []
zpos = []
atom = []

axpos = []
aypos = []
azpos = []
aatom = []


n = 1
while n < 1024:
    aatom.append(alines[n-1])
    axpos.append(float(alines[n]))
    aypos.append(float(alines[n+1]))
    azpos.append(float(alines[n+2]))
    n += 4

######
# Now multiply out atom positions into Angstroms
######

n = 0 
while n < 256:
    axpos[n] = axpos[n]*alat_ang
    aypos[n] = aypos[n]*alat_ang
    azpos[n] = azpos[n]*alat_ang
    n += 1


######
# Now make a list of charges for each atom
######
charge = []
n = 0
while n < 256:
    if float(aatom[n]) == 1:
        charge.append(78)
    if float(aatom[n]) == 2:
        charge.append(28)
    if float(aatom[n]) == 3:
        charge.append(15)
    n += 1


# Now calculate distance doing loop
# since cell is 3-D there are 9 periodic cells around the box which means that per atom there are 10 distance calculations done
# num will be number of atoms * 10
# Need two loops 1 loop is for each atom, the second loop 


# Empty lists that will be used
# to compute x, y and z components 
xtemp  = []
ytemp  = []
ztemp  = []


xfield = []
yfield = []
zfield = []

#####
# Beginning of Indicie List
#####

# We can calculate new x2, y2 and z2 values 
# Now make a list of all possible combinations
i = []
j = []
k = []

indicies = [0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1]
#all negative indicies
indicies.extend([-1,0,0,0,-1,0,0,0,-1,-1,-1,0,-1,0,-1,0,-1,-1,-1,-1,-1])
# now we need mixed indicies 
indicies.extend([-1,-1,1,-1,1,-1,1,-1,-1])
indicies.extend([-1,1,1,1,-1,1,1,1,-1])
indicies.extend([0,-1,1,0,1,-1,-1,0,1,-1,1,0])
   
n = 0  
while n < 75:
    # print(str(indicies[n])+' '+str(indicies[n+1])+' '+str(indicies[n+2]))
    i.append(indicies[n])
    j.append(indicies[n+1])
    k.append(indicies[n+2])
    n += 3
    
#####
# END OF INDICIE LIST
#####

# Now do distance calculation changing x2, y2 and z2 based on i, j and k indicies 
# We need multiple loops to execute the distance calculation
# outermost loop is to range over atoms r 
# next loop over atoms m
# final loop is to calculate atom m + images

# Initially we had set up to avoid double counting, however double counting is actually necesasry 


while r <= (atom_num-1):
    m = 0
    while (m) <= (atom_num-1):
        n = 0
        while n < (25):
                x1 = axpos[r]
                y1 = aypos[r]
                z1 = azpos[r]
                x2 = axpos[m]+i[n]*a
                y2 = aypos[m]+j[n]*b
                z2 = azpos[m]+k[n]*c
                dist_image = np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)
                if dist_image < cutoff and r != m:
                    # now use if statements to choose which atom types and get charge information
                    # We will compute comlumbic force contributions for each x, y and z component
                    force = (colum*e2*charge[r]*charge[m])/(dist_image)**2
                    xtemp.append(force*(x1-x2))
                    ytemp.append(force*(y1-y2))
                    ztemp.append(force*(z1-z2))
                    print('Distance is less then cutoff')
                    print(dist_image)
                    #print off indicies for the PBE cells and Atom Index
                    print('Atom Index A:',r,'Atom Index B:',(m))
                    #print(str(i[n])+ ' '+str(j[n])+ ' '+ str(k[n])) 
                    print('\n')                
                n += 1
        m += 1
    if len(xtemp) != 0:
        columb_force.append(force)
        xfield.append(np.sum(xtemp))
        yfield.append(np.sum(ytemp))
        zfield.append(np.sum(ztemp))
        xtemp = []
        ytemp = []
        ztemp = []
    r += 1

######
# End of the Calculation
######

Distance is less then cutoff
2.37628478009
Atom Index A: 0 Atom Index B: 4


Distance is less then cutoff
2.7682852824
Atom Index A: 0 Atom Index B: 5


Distance is less then cutoff
2.88642076473
Atom Index A: 0 Atom Index B: 6


Distance is less then cutoff
2.79999734138
Atom Index A: 0 Atom Index B: 7


Distance is less then cutoff
2.93289985636
Atom Index A: 0 Atom Index B: 23


Distance is less then cutoff
2.94941639372
Atom Index A: 0 Atom Index B: 31


Distance is less then cutoff
2.77259142903
Atom Index A: 0 Atom Index B: 35


Distance is less then cutoff
2.52936191082
Atom Index A: 0 Atom Index B: 36


Distance is less then cutoff
3.28304539162
Atom Index A: 0 Atom Index B: 54


Distance is less then cutoff
3.26311913759
Atom Index A: 0 Atom Index B: 64


Distance is less then cutoff
3.37706816091
Atom Index A: 0 Atom Index B: 190


Distance is less then cutoff
2.56580427758
Atom Index A: 0 Atom Index B: 191


Distance is less then cutoff
2.73609091297
Atom Index A: 0 Atom Ind

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy

######
# Prepare Lammps file with Vectors
######

######
# Prepare Lammps File
######

# We are going to print out the final structure with an additional column for distance moved
text = []

n = 0
while n < 256:
    text.append((str(nums[1+n])+' '+str(aatom[n])+' '+str(axpos[n])+ ' '+str(aypos[n])+' '+str(azpos[n])+' '+str(xfield[n])+' '+str(yfield[n])+' '+str(zfield[n])+' '+str(columb_force[n])))
    n += 1  

#####
# Top formatting
#####  
newfilename = 'sep23_1_columb_force'

file = open(newfilename, "w")
# First 3 write lines are to show size of simulation box
file.write('ITEM: TIMESTEP')
file.write('\n'+'0')
file.write('\n'+'ITEM: NUMBER OF ATOMS')
file.write('\n'+'256')
file.write('\n'+'ITEM: BOX BOUNDS pp pp pp')
file.write('\n'+'0'+' '+ str(alat_ang))
file.write('\n'+'0'+' '+str(alat_ang))
file.write('\n'+'0'+' '+str(alat_ang))
file.write('\n'+'ITEM: ATOMS id type x y z fx fy fz force \n')

#####
# Write out text to the end of file
#####

with open(newfilename, 'w') as f:
    for row in text:
        file.write("%s\n" % str(row))
        
file.close()

In [1]:
# Comment Line 

import math
import numpy as np
import matplotlib.pyplot as plt
import scipy

# First we need to define the lattice length
# next multiply by the a constant to convert to angstrom
# Nums is the number of atoms
alat = 29.00
alat_ang = alat*0.529
nums = list(range(4))

atom_num = 3
dist = []

# Coulumb Constant : Units : N m^2 C^-2
colum = 9.0*10**29
e2 = (1.602*10**-19)**2


# lattice lengths a for x, b for y, c for z
a = 15.341
b = 15.341
c = 15.341

cutoff = 3.5

######
# Use float to convert to float numbers
######

xpos = []
ypos = []
zpos = []
atom = []

axpos = []
aypos = []
azpos = []
aatom = []


n = 1
while n < 12:
    aatom.append(alines[n-1])
    axpos.append(float(alines[n]))
    aypos.append(float(alines[n+1]))
    azpos.append(float(alines[n+2]))
    n += 4


######
# Now make a list of charges for each atom
######
charge = []
n = 0
while n < 3:
    if float(aatom[n]) == 1:
        charge.append(78)
    if float(aatom[n]) == 2:
        charge.append(28)
    if float(aatom[n]) == 3:
        charge.append(15)
    n += 1


# Now calculate distance doing loop
# since cell is 3-D there are 9 periodic cells around the box which means that per atom there are 10 distance calculations done
# num will be number of atoms * 10
# Need two loops 1 loop is for each atom, the second loop 


# Empty lists that will be used
# to compute x, y and z components 
xtemp  = []
ytemp  = []
ztemp  = []


xfield = []
yfield = []
zfield = []

#####
# Beginning of Indicie List
#####

# We can calculate new x2, y2 and z2 values 
# Now make a list of all possible combinations
i = []
j = []
k = []

indicies = [0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1]
#all negative indicies
indicies.extend([-1,0,0,0,-1,0,0,0,-1,-1,-1,0,-1,0,-1,0,-1,-1,-1,-1,-1])
# now we need mixed indicies 
indicies.extend([-1,-1,1,-1,1,-1,1,-1,-1])
indicies.extend([-1,1,1,1,-1,1,1,1,-1])
indicies.extend([0,-1,1,0,1,-1,-1,0,1,-1,1,0])
   
n = 0  
while n < 75:
    # print(str(indicies[n])+' '+str(indicies[n+1])+' '+str(indicies[n+2]))
    i.append(indicies[n])
    j.append(indicies[n+1])
    k.append(indicies[n+2])
    n += 3
    
#####
# END OF INDICIE LIST
#####

# Now do distance calculation changing x2, y2 and z2 based on i, j and k indicies 
# We need multiple loops to execute the distance calculation
# outermost loop is to range over atoms r 
# next loop over atoms m
# final loop is to calculate atom m + images
m = 0
r = 0
x = 0
cluster = []
averages = []
# Initially we had set up to avoid double counting, however double counting is actually necesasry 


while r <= (atom_num-1):
    m = 0
    while (m) <= (atom_num-1):
        n = 0
        while n < (25):
                x1 = axpos[r]
                y1 = aypos[r]
                z1 = azpos[r]
                x2 = axpos[m]+i[n]*a
                y2 = aypos[m]+j[n]*b
                z2 = azpos[m]+k[n]*c
                dist_image = np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)
                if dist_image < cutoff and r != m:
                    # now use if statements to choose which atom types and get charge information
                    # We will compute comlumbic force contributions for each x, y and z component
                    xtemp.append((colum*e2*charge[r]*charge[m])/(axpos[r]-axpos[m])**2)
                    ytemp.append((colum*e2*charge[r]*charge[m])/(aypos[r]-aypos[m])**2)
                    ztemp.append((colum*e2*charge[r]*charge[m])/(azpos[r]-azpos[m])**2)
                    #print('Distance is less then cutoff')
                    #print(dist_image)
                    #print off indicies for the PBE cells and Atom Index
                    #print('Atom Index A:',r,'Atom Index B:',(m))
                    #print(str(i[n])+ ' '+str(j[n])+ ' '+ str(k[n])) 
                    #print('\n')                
                n += 1
        m += 1
    if len(xtemp) != 0:
        xfield.append(np.sum(xtemp))
        yfield.append(np.sum(ytemp))
        zfield.append(np.sum(ztemp))
        xtemp = []
        ytemp = []
        ztemp = []
    r += 1

######
# End of the Calculation
######

NameError: name 'alines' is not defined

In [ ]:
# Comment Line 
# Electric Field Calculation
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy
import time

start = time.time()

# First we need to define the lattice length
# next multiply by the a constant to convert to angstrom
# Nums is the number of atoms
alat = 29.00
alat_ang = alat*0.529
atom_num = 256

# Coulumb Constant : Units : N m^2 C^-2
colum = 9.0*10**29
e2 = (1.602*10**-19)**2

# lattice lengths a for x, b for y, c for z
a = 15.341
b = 15.341
c = 15.341

cutoff_out = 3.0
cutoff_in  = 1.0

######
# Create all empty lists
######
xpos,ypos,zpos,atom,axpos,aypos,azpos,aatom,charge = ([] for i in range(9))
xtemp,ytemp,ztemp,xfield,yfield,zfield,i,j,k = ([] for i in range(9))


n = 1
while n < (atom_num*4):
    aatom.append(alines[n-1])
    axpos.append(float(alines[n]))
    aypos.append(float(alines[n+1]))
    azpos.append(float(alines[n+2]))
    n += 4


######
# Now make a list of charges for each atom
######
n = 0
while n < atom_num:
    if float(aatom[n]) == 1:
        charge.append(78)
    if float(aatom[n]) == 2:
        charge.append(28)
    if float(aatom[n]) == 3:
        charge.append(15)
    n += 1

#####
# Beginning of Indicie List
#####

# We can calculate new x2, y2 and z2 values 
# Now make a list of all possible combinations

indicies = [0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,1,0,1,1,1,1,1]
#all negative indicies
indicies.extend([-1,0,0,0,-1,0,0,0,-1,-1,-1,0,-1,0,-1,0,-1,-1,-1,-1,-1])
# now we need mixed indicies 
indicies.extend([-1,-1,1,-1,1,-1,1,-1,-1])
indicies.extend([-1,1,1,1,-1,1,1,1,-1])
indicies.extend([0,-1,1,0,1,-1,-1,0,1,-1,1,0])
   
n = 0  
while n < 75:
    # print(str(indicies[n])+' '+str(indicies[n+1])+' '+str(indicies[n+2]))
    i.append(indicies[n])
    j.append(indicies[n+1])
    k.append(indicies[n+2])
    n += 3
    
#####
# END OF INDICIE LIST
#####

# Now do distance calculation changing x2, y2 and z2 based on i, j and k indicies 
# We need multiple loops to execute the distance calculation
# outermost loop is to range over atoms r 
# next loop over atoms m
# final loop is to calculate atom m + images
 
x = np.arange(0,15.341,.192)
y = np.arange(0,15.341,.192)
z = np.arange(0,15.341,.192)
check_list = []

l = 0
# x-loop
while l < len(x):
    # x-loop
    m = 0
    while m < len(y):
        # y-loop
        n = 0
        while n < len(z):
            # z-loop
            o = 0
            while o <= (atom_num-1):
                # atom-loop
                p = 0
                while p < 25:
                    #periodic images loop
                    x1 = axpos[o]
                    y1 = aypos[o]
                    z1 = azpos[o]
                    x2 = x[l]
                    y2 = y[m]
                    z2 = z[n]
                    dist_image = np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)
                    if dist_image < cutoff_out and dist_image > cutoff_in:
                        xtemp.append((colum*e2*charge[o])/(axpos[o]-x[l])**2)
                        ytemp.append((colum*e2*charge[o])/(aypos[o]-y[m])**2)
                        ztemp.append((colum*e2*charge[o])/(azpos[o]-z[n])**2)
                        check_list.append(1)
                    p += 1
                o += 1
            # Before we move to new coordinate 
            # assign all temp fields
            if len(xtemp) != 0:
                xfield.append(np.sum(xtemp))
                yfield.append(np.sum(ytemp))
                zfield.append(np.sum(ztemp))
                xtemp = []
                ytemp = []
                ztemp = []
            else:
                xfield.append(0)
                yfield.append(0)
                zfield.append(0)
            n += 1
        m += 1
    l += 1
    
######
# End of the Calculation
######
end = time.time()
print("Calculation time is:",end-start)

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy

print("Hello")